In [5]:
!pip install pandas
!pip install scikit-learn
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.1 MB 13.4 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.1 MB 14.9 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.1 MB 14.9 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.1 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   -- ------------------------------------- 2.9/44.5 MB 13.9 MB/s eta 0:00:03
   ------ --------------------------------- 6.8/44.5 MB 16.1 MB/s eta 0:00:03
   --------- ------------------------------ 10.7/44.5 MB 17.2 MB/s eta 0:00:02

In [7]:
import pandas as pd
import numpy as np

In [15]:
dataset = pd.read_csv('All_teams_WHL_stats.csv')
target_team = 'Spokane Chiefs'
k = 5 # trained on the last k games

In [12]:
dataset = dataset.sort_values(by="Game_ID")
display(dataset)

,Unnamed: 0,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
5495,5495,1014692,Prince George Cougars,Calgary Hitmen,7,1,0.285714,0.250000,34,17,0.433333,0.566667
5496,5496,1014699,Kamloops Blazers,Calgary Hitmen,2,4,0.000000,0.000000,32,56,0.492958,0.507042
5497,5497,1014708,Kelowna Rockets,Calgary Hitmen,3,4,0.166667,0.000000,34,26,0.461538,0.538462
5498,5498,1014720,Calgary Hitmen,Lethbridge Hurricanes,1,3,0.000000,0.285714,31,37,0.500000,0.500000
5499,5499,1014735,Red Deer Rebels,Calgary Hitmen,4,2,0.000000,0.000000,31,26,0.576271,0.423729
...,...,...,...,...,...,...,...,...,...,...,...,...
2568,2568,1021542,Kelowna Rockets,Prince George Cougars,3,5,0.000000,1.000000,20,34,0.464286,0.535714
618,618,1021543,Portland Winterhawks,Spokane Chiefs,6,4,0.333333,0.500000,47,30,0.677419,0.322581
3669,3669,1021544,Red Deer Rebels,Swift Current Broncos,4,5,0.250000,0.250000,33,35,0.514706,0.485294
5494,5494,1021545,Calgary Hitmen,Prince Albert Raiders,8,5,0.333333,0.250000,44,23,0.424242,0.575758


In [20]:
# Get default values for averages of goals, PP%, SOG, and FOW%
default_goals = (dataset['Home_Goals'].mean() + dataset['Away_Goals'].mean()) // 2
default_ppp = (dataset['Home_PP%'].mean() + dataset['Away_PP%'].mean()) / 2
default_sog = (dataset['Home_SOG'].mean() + dataset['Away_SOG'].mean()) // 2
default_fowp = (dataset['Home_FOW%'].mean() + dataset['Away_FOW%'].mean()) / 2

print(default_goals, default_ppp, default_sog, default_fowp)

3.0 0.2200645699906055 31.0 0.5


In [27]:
def get_opponent_stats(dataset, queue):
  opponent_goals = 0
  opponent_ppp = 0
  opponent_fowp = 0
  opponent_sog = 0
  # Get the last k games averages
  # This creates a new dataset of only opponent games, and which are before the current game
  opponent_games = dataset[
    ((dataset['Home_Name'] == opponent_name) | (dataset['Away_Name'] == opponent_name)) & 
    (dataset['Game_ID'] < row['Game_ID'])
  ].tail(k)
  # If the opponent hasnt played k games yet, use default
  if len(opponent_games) < k:
    opponent_goals = default_goals
    opponent_ppp = default_ppp
    opponent_fowp = default_fowp
    opponent_sog = default_sog
  else:
    # Get the mean of the entire column that the opponent played
    opponent_goals = opponent_games['Home_Goals'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_Goals']).mean()
    opponent_ppp = opponent_games['Home_PP%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_PP%']).mean()
    opponent_fowp = opponent_games['Home_FOW%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_FOW%']).mean()
    opponent_sog = opponent_games['Home_SOG'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_SOG']).mean()

  return opponent_goals, opponent_ppp, opponent_fowp, opponent_sog

In [28]:
from collections import deque
target_queue = deque(maxlen=k)
target_df = pd.DataFrame(columns = [
    "target_goals", 
    "opponent_goals", 
    "target_PP%", 
    "opponent_PP%", 
    "target_SOG", 
    "opponent_SOG",
    "target_FOW%",
    "opponent_FOW%",
    "Home/Away",
    "target_win"
])

target_goals = 0
target_ppp = 0
target_fowp = 0
target_sog = 0

# Iterate through the DataFrame to track the last 5 games
for index, row in dataset.iterrows():
    if row['Home_Name'] == target_team or row['Away_Name'] == target_team:

      side = 'Home' if row['Home_Name'] == target_team else 'Away'
      opponent_name = row['Away_Name'] if side == 'Home' else row['Home_Name']
      # now we are within the target team
      # If the queue is less than k, just add to queue
      if len(target_queue) < k:
        target_goals += row[side + '_Goals']
        target_ppp += row[side + '_PP%']
        target_fowp += row[side + '_FOW%']
        target_sog += row[side + '_SOG']
        target_queue.append(row.to_dict())
      else: #If queue is full, then we can do some popping

        opponent_goals, opponent_ppp, opponent_fowp, opponent_sog = get_opponent_stats(dataset, target_queue)
        
        # print(target_goals / k, target_ppp / k, target_fowp / k, target_sog / k)
        # print(opponent_goals, opponent_ppp, opponent_fowp, opponent_sog)
        
        # target_df[len(target_df)] = {
          
        # }
        
        # target_queue.append(row.to_dict())  # Add the game to the queue
        
